In [1]:
import pandas as pd
from langdetect import detect
import glob



In [21]:
all_files = glob.glob("./annotations/*.tsv")

li = []

for filename in all_files:

    df = pd.read_table(filename, index_col=None, header=0, sep='\t')
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [22]:
frame

,tweet_id,image_id,text_info,text_info_conf,image_info,image_info_conf,text_human,text_human_conf,image_human,image_human_conf,image_damage,image_damage_conf,tweet_text,image_url,image_path
0,917791044158185473,917791044158185473_0,informative,1.0000,informative,0.6766,other_relevant_information,1.0000,other_relevant_information,0.6766,NaN,NaN,RT @Gizmodo: Wildfires raging through Northern...,http://pbs.twimg.com/media/DLyi_WYVYAApwNg.jpg,data_image/california_wildfires/10_10_2017/917...
1,917791130590183424,917791130590183424_0,informative,1.0000,informative,0.6667,infrastructure_and_utility_damage,1.0000,affected_individuals,0.6667,NaN,NaN,PHOTOS: Deadly wildfires rage in California ht...,http://pbs.twimg.com/media/DLymKm9UMAAu0qw.jpg,data_image/california_wildfires/10_10_2017/917...
2,917791291823591425,917791291823591425_0,informative,0.6813,informative,1.0000,other_relevant_information,0.6813,infrastructure_and_utility_damage,1.0000,severe_damage,1.0,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaaZV4AAjT7x.jpg,data_image/california_wildfires/10_10_2017/917...
3,917791291823591425,917791291823591425_1,informative,0.6813,not_informative,1.0000,other_relevant_information,0.6813,not_humanitarian,1.0000,NaN,NaN,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,http://pbs.twimg.com/media/DLudaZXUMAABAEZ.jpg,data_image/california_wildfires/10_10_2017/917...
4,917792092100988929,917792092100988929_0,informative,0.6727,informative,0.6612,other_relevant_information,0.6727,infrastructure_and_utility_damage,0.6612,severe_damage,1.0,RT @TIME: California's raging wildfires as you...,http://pbs.twimg.com/media/DLwNe-NXUAE0XCw.jpg,data_image/california_wildfires/10_10_2017/917...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18077,916105333872918528,916105333872918528_0,informative,1.0000,not_informative,1.0000,rescue_volunteering_or_donation_effort,1.0000,not_humanitarian,1.0000,NaN,NaN,Carlos santana donates $100k to mexico earthqu...,http://pbs.twimg.com/media/DLao8g8W4AAKoPV.jpg,data_image/mexico_earthquake/6_10_2017/9161053...
18078,916105374134042624,916105374134042624_0,informative,1.0000,not_informative,0.6412,rescue_volunteering_or_donation_effort,1.0000,not_humanitarian,0.6412,NaN,NaN,Seth troxler donates 10k to earthquake relief ...,http://pbs.twimg.com/media/DLao-2GWsAA97vC.jpg,data_image/mexico_earthquake/6_10_2017/9161053...
18079,916112796194021376,916112796194021376_0,informative,1.0000,informative,0.6337,rescue_volunteering_or_donation_effort,1.0000,other_relevant_information,0.6337,NaN,NaN,Entercom/San Francisco Stations Raise Funds Fo...,http://pbs.twimg.com/media/DLavu1YV4AAgSJ_.jpg,data_image/mexico_earthquake/6_10_2017/9161127...
18080,916164076484653056,916164076484653056_0,informative,0.6580,informative,0.6374,other_relevant_information,0.6580,rescue_volunteering_or_donation_effort,0.6374,NaN,NaN,Mexico Earthquakes | International Medical Cor...,http://pbs.twimg.com/media/DLbeXxlWsAEl9xu.jpg,data_image/mexico_earthquake/6_10_2017/9161640...


In [23]:
frame.columns

Index(['tweet_id', 'image_id', 'text_info', 'text_info_conf', 'image_info',
       'image_info_conf', 'text_human', 'text_human_conf', 'image_human',
       'image_human_conf', 'image_damage', 'image_damage_conf', 'tweet_text',
       'image_url', 'image_path'],
      dtype='object')

In [24]:
frame = frame.drop_duplicates(subset='tweet_id', keep="first")

In [25]:
len(frame)

16058

In [26]:
frame.drop(frame.columns.difference(['tweet_id','text_info','tweet_text']), 1, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
frame.columns=["id","Informativeness","text"]

In [29]:
frame.Informativeness.value_counts()

informative        11503
not_informative     4555
Name: Informativeness, dtype: int64

In [30]:
### Keep only english tweets

def is_english(text):

    try:
          language = detect(text)
    except:
          #These will be removed
          print("This row throws and error:", text)
          return False;
    if(language=='en'):
      return True;
    else:
      return False;

frame_en=frame[frame.apply(lambda x: is_english(x['text']), axis=1)]

In [31]:
len(frame_en)

15745

In [32]:
frame_en.Informativeness.value_counts()

informative        11365
not_informative     4380
Name: Informativeness, dtype: int64

In [33]:
frame_en.head()

,id,Informativeness,text
0,917791044158185473,informative,RT @Gizmodo: Wildfires raging through Northern...
1,917791130590183424,informative,PHOTOS: Deadly wildfires rage in California ht...
2,917791291823591425,informative,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...
4,917792092100988929,informative,RT @TIME: California's raging wildfires as you...
5,917792147700465664,informative,Wildfires Threaten Californiaâ€™s First Legal ...


In [34]:
frame_en['Informativeness']=frame_en['Informativeness'].map(lambda x: 1 if(x=='informative') else 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
frame_en["Informativeness"].value_counts()

1    11365
0     4380
Name: Informativeness, dtype: int64

In [93]:
frame_en.to_pickle("./../dataset2Cleaned.pkl")

In [19]:
users=pd.read_csv('./../tj/parsed/twitter_user.csv')

In [45]:
authors=pd.read_csv('./../tj/parsed/tweet_metadata.csv')

In [67]:
merged=pd.merge(frame_en, authors, on='id')

In [68]:
merged.drop(merged.columns.difference(['id', 'Informativeness', 'text', 'author_id', 'tweet_type', 'retweet_count', 'favorite_count']), 1, inplace=True)

In [69]:
merged.head()

,id,Informativeness,text,author_id,tweet_type,retweet_count,favorite_count
0,917791130590183424,1,PHOTOS: Deadly wildfires rage in California ht...,23085284,tweet,0,0
1,917791291823591425,1,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,765553757887266816,retweet,497,0
2,917792092100988929,1,RT @TIME: California's raging wildfires as you...,1143801686,retweet,197,0
3,917792147700465664,1,Wildfires Threaten Californiaâ€™s First Legal ...,743291168092364805,tweet,0,0
4,917793158251077632,1,RT @FoxNews: Southern California fire shrouds ...,69611563,retweet,66,0


In [70]:
users.head()

,author_id,has_description,bio_has_url,followers_count,friends_count,favourites_count,listed_count,statuses_count,protected,verified,default_profile,default_profile_image
0,6925932,娘魔の3歳です。仲良くやってます・・・(笑),NaN,84,555,42060,0.0,2893,False,False,False,False
1,820569,Author of THIS TIME THE FIRE and UNINVENTABLES...,http://t.co/mpmrJeB7hX,682,1487,4989,29.0,9394,False,False,False,False
2,4926751,Life's how you develop it.|Papa van 💁‍♀️💁‍♂️🙅‍...,https://t.co/uas8e6LpeN,1181,956,2750,37.0,41861,False,False,False,False
3,6013032,NaN,NaN,0,0,0,0.0,23774,False,False,False,True
4,5916622,なー！起きるのと片付けが苦手。惰眠に惨敗な人生、そんな自分に甘えまくり好きな事ばかりしてる。...,http://t.co/ntGHFHbqPF,119,203,586,7.0,8767,False,False,False,False


In [64]:
users.columns

Index(['id', 'description', 'url', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'statuses_count', 'protected',
       'verified', 'default_profile', 'default_profile_image'],
      dtype='object')

In [62]:
users.drop(columns=['created_at', 'lang', 'name', 'screen_name', 'location','access'], inplace=True)

In [65]:
users.columns=['author_id', 'has_description', 'bio_has_url', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'statuses_count', 'protected',
       'verified', 'default_profile', 'default_profile_image']

In [71]:
merged=pd.merge(merged, users, on='author_id')

In [90]:
pd.read_csv('./../tj/parsed/view_user_stats.csv')

,author_id,tw_num,do_rt_num,get_rt_num
0,1000002343,1,0,0
1,1000018914,1,1,0
2,1000023236,1,0,0
3,1000032338,1,1,0
4,100003468,1,1,0
...,...,...,...,...
241959,999982998,1,0,0
241960,99998316,1,0,0
241961,999985153,1,1,0
241962,99999199,1,1,0


In [91]:
merged

,id,Informativeness,text,author_id,tweet_type,retweet_count,favorite_count,has_description,bio_has_url,followers_count,friends_count,favourites_count,listed_count,statuses_count,protected,verified,default_profile,default_profile_image
0,917791130590183424,1,PHOTOS: Deadly wildfires rage in California ht...,23085284,tweet,0,0,Real-time updates from WMTW News 8: Maine's To...,https://t.co/hJRuLYj5zC,87310,417,213,692.0,270112,False,True,False,False
1,914624000625541121,1,Family helps community by cooking meals for do...,23085284,tweet,0,0,Real-time updates from WMTW News 8: Maine's To...,https://t.co/hJRuLYj5zC,87310,417,213,692.0,270112,False,True,False,False
2,917791291823591425,1,RT @Cal_OES: PLS SHARE: Weâ€™re capturing wild...,765553757887266816,retweet,497,0,Official Twitter account of Alerts Made Easy®....,https://t.co/NTpDIXXuLv,63,217,2994,4.0,3864,False,False,True,False
3,917792092100988929,1,RT @TIME: California's raging wildfires as you...,1143801686,retweet,197,0,Anarchist furry trans pan degenerate that play...,NaN,71,146,15903,1.0,6560,False,False,False,False
4,917792147700465664,1,Wildfires Threaten Californiaâ€™s First Legal ...,743291168092364805,tweet,0,0,Getting out the latest information in the Cann...,NaN,11350,366,180,87.0,76350,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12412,915972133746040833,1,Mexico City's Earthquake Alert Worked. The Res...,852588989659963393,tweet,1,1,My name is Justin Habana. Self proclaimed Huma...,NaN,710,492,11,2.0,1066,False,False,True,False
12413,915972197319155712,0,Photos that will make you want to Visit Mexico...,6660222,tweet,0,0,Video Consultant: https://t.co/hfQDKOjjWC OG t...,https://t.co/6Hts4agQXc,6280,3128,7670,488.0,36269,False,False,False,False
12414,916075589131436032,1,A Wave Of Natural Disasters StrikeÂ Mexico htt...,862739444419493889,tweet,0,0,"Dedicated to delivering the most premium, non-...",https://t.co/Yk6kTj1A6T,203,3049,188,4.0,243,False,False,True,False
12415,916164076484653056,1,Mexico Earthquakes | International Medical Cor...,10118982,tweet,0,0,"Single, 53, no children. Let's talk about it o...",https://t.co/WXpZYqAQbU,2978,4998,228939,435.0,426955,False,False,False,False
